Skript zum Scrapen der Kanal infos. Jeder Kanal hat seine eigene JSON Datei in "channel_data"

beispiel Datensatz:

    {
        "channel_name": "(G)I-DLE (여자)아이들 (Official YouTube Channel)",
        "subs": "8,47 Mio.",
        "views": "5.098.358.931",
        "num_videos": "1.198",
        "created_at": "19.04.2018",
        "country": "Südkorea",
        "timestamp": "2024-07-08_23-36-16"
    }

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common import TimeoutException, StaleElementReferenceException
from datetime import datetime
from utils import extract_date
from tqdm import tqdm  # Import tqdm for progress bar

import json
import os

Cookies akzeptieren

In [2]:
chrome_options = webdriver.ChromeOptions()
# Addon das Dislike Anzahl anzeigt
chrome_options.add_extension('./add_dislike_addon.crx')
# macht Browser unsichtbar
# chrome_options.add_argument('--headless=new')

driver = webdriver.Chrome(options=chrome_options)  
driver.get('https://www.youtube.com/feed/trending')

try:
    accept_all = driver.find_element(By.XPATH, '/html/body/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/form[2]/div/div/button')

    accept_all.click()
except TimeoutException:
    print('Cookie Modal not found')

Logik zum Scrapen der Kanal Infos

In [3]:
def scrape_channel_data(driver, channel_url):
    driver.get(channel_url)
    time.sleep(3)
    try:
        WebDriverWait(driver, 30).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, '#page-header'))
        )
        channel_data = {}
        channel_data['channel_name'] = driver.find_element(By.CSS_SELECTOR, '#page-header h1').text
        driver.find_element(By.CSS_SELECTOR, '.yt-description-preview-view-model-wiz  .truncated-text-wiz__absolute-button').click()

        WebDriverWait(driver, 30).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'div #about-container'))
        )

        channel_details = driver.find_elements(By.CSS_SELECTOR, 'tr.description-item')
        channel_data['subs'] = channel_details[3].find_elements(By.TAG_NAME, 'td')[1].text.replace(' Abonnenten', '')
        channel_data['views'] = channel_details[5].find_elements(By.TAG_NAME, 'td')[1].text.replace(' Aufrufe', '')
        channel_data['num_videos'] = channel_details[4].find_elements(By.TAG_NAME, 'td')[1].text.replace(' Videos', '')
        channel_data['created_at'] = extract_date(channel_details[6].find_elements(By.TAG_NAME, 'td')[1].text)
        channel_data['country'] = channel_details[7].find_elements(By.TAG_NAME, 'td')[1].text

        return channel_data
    except Exception as e:
        print(f"Error scraping data for {channel_url}: {str(e)}")
        return None


schaut nach allen einzigartigen Kanälen die jemals von den Trends gescraped wuden. ruft füre jeden kanal die scrape funktion auf

In [4]:
def track_channel_growth(unique_channels_file, output_folder):
    with open(unique_channels_file, 'r', encoding='utf-8') as file:
        unique_channels = json.load(file)
    
    # channel_url - schlüssel;
    for channel_url, channel_name in tqdm(unique_channels.items(), desc="Tracking Channels"):
        #bereinigt den Channel namen damit man ihn als Dateinamen nutzen kann
        channel_name = channel_name['channel_name'].replace(' ', '_')
        cleaned_channel_name = ''.join(c if c.isalnum() else '_' for c in channel_name)

        file_path = os.path.join(output_folder, f'{cleaned_channel_name}.json')
        
        if os.path.exists(file_path):
            with open(file_path, 'r', encoding='utf-8') as file:
                existing_data = json.load(file)
        else:
            existing_data = []
        
        channel_data = scrape_channel_data(driver, channel_url)
        if channel_data:
            date_str = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
            channel_data['timestamp'] = date_str
            existing_data.append(channel_data)
            
            try:
                with open(file_path, 'w', encoding='utf-8') as file:
                    json.dump(existing_data, file, ensure_ascii=False, indent=4)
                    print(f"Data saved to {file_path}")
            except Exception as e:
                print(f"Error saving {file_path}: {str(e)}")


In [5]:
unique_channels_file = 'unique_channels/unique_channels.json'
output_folder = 'channel_data'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

tqdm.display
track_channel_growth(unique_channels_file, output_folder)
driver.quit()

Tracking Channels:   2%|▏         | 1/42 [00:15<10:50, 15.88s/it]

Data saved to channel_data\DAVE.json


Tracking Channels:   5%|▍         | 2/42 [00:21<06:31,  9.78s/it]

Data saved to channel_data\Rinos.json


Tracking Channels:   7%|▋         | 3/42 [00:29<05:43,  8.82s/it]

...

Data saved to channel_data\Y_Kollektiv.json


Tracking Channels:  98%|█████████▊| 41/42 [06:02<00:07,  7.43s/it]

Data saved to channel_data\Critical_Role.json


Tracking Channels: 100%|██████████| 42/42 [06:11<00:00,  8.83s/it]

Data saved to channel_data\Joshinator.json
